In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import seaborn as sns
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

In [ ]:
# df = pd.read_csv('/content/drive/My Drive/SHQ/Data/cfa_test_df.csv')

df = pd.read_csv('/content/drive/My Drive/SHQ/Data/cfa_test_df2.csv')

In [ ]:
df

,age,gender,hand,navigating_skills,sleep,edu_level,commute_level,home_env
0,68.0,0,0,1,8.0,0,0,2
1,18.0,1,1,2,7.0,1,0,1
2,18.0,1,1,3,7.0,1,2,2
3,21.0,0,0,2,7.0,0,1,1
4,36.0,1,0,3,5.0,0,1,0
...,...,...,...,...,...,...,...,...
909451,50.0,1,1,2,6.0,1,2,0
909452,68.0,1,0,2,8.0,0,1,2
909453,59.0,0,0,3,7.0,1,0,2
909454,48.0,1,0,2,6.0,0,2,1


In [ ]:
df.head()

,age,gender,hand,navigating_skills,sleep,edu_level,commute_level,home_env
0,68.0,0,0,1,8.0,0,0,2
1,18.0,1,1,2,7.0,1,0,1
2,18.0,1,1,3,7.0,1,2,2
3,21.0,0,0,2,7.0,0,1,1
4,36.0,1,0,3,5.0,0,1,0


## Step1. Clustering


In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.preprocessing import scale,RobustScaler,StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
#import mglearn
import matplotlib.pyplot as plt

In [ ]:
# MinMax scaled
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(df)
df2 = scaler.transform(df)
df2 = pd.DataFrame(df2,index=df.index,columns=['age',	'gender',	'hand',	'navigating_skills',	'sleep',	'edu_level',	'commute_level',	'home_env'])
df2[:10]

,age,gender,hand,navigating_skills,sleep,edu_level,commute_level,home_env
0,0.961538,0.0,0.0,0.333333,0.636364,0.0,0.0,1.0
1,0.000000,1.0,1.0,0.666667,0.545455,1.0,0.0,0.5
2,0.000000,1.0,1.0,1.000000,0.545455,1.0,1.0,1.0
3,0.057692,0.0,0.0,0.666667,0.545455,0.0,0.5,0.5
4,0.346154,1.0,0.0,1.000000,0.363636,0.0,0.5,0.0
5,0.019231,1.0,0.0,0.666667,0.636364,1.0,0.0,0.5
6,0.826923,1.0,0.0,0.666667,0.545455,0.0,0.0,0.5
7,0.211538,1.0,0.0,1.000000,0.545455,0.0,1.0,0.0
8,0.019231,0.0,0.0,0.333333,0.636364,0.0,0.0,0.5
9,0.384615,1.0,0.0,0.666667,0.545455,0.0,0.0,0.0


In [ ]:
test = scaler.transform(df)

In [ ]:
df2.head()

,age,gender,hand,navigating_skills,sleep,edu_level,commute_level,home_env
0,0.961538,0.0,0.0,0.333333,0.636364,0.0,0.0,1.0
1,0.000000,1.0,1.0,0.666667,0.545455,1.0,0.0,0.5
2,0.000000,1.0,1.0,1.000000,0.545455,1.0,1.0,1.0
3,0.057692,0.0,0.0,0.666667,0.545455,0.0,0.5,0.5
4,0.346154,1.0,0.0,1.000000,0.363636,0.0,0.5,0.0


In [ ]:
df2 = df2.drop(columns="navigating_skills", axis=1)

In [ ]:
df2.head()

,age,gender,hand,sleep,edu_level,commute_level,home_env
0,0.961538,0.0,0.0,0.636364,0.0,0.0,1.0
1,0.000000,1.0,1.0,0.545455,1.0,0.0,0.5
2,0.000000,1.0,1.0,0.545455,1.0,1.0,1.0
3,0.057692,0.0,0.0,0.545455,0.0,0.5,0.5
4,0.346154,1.0,0.0,0.363636,0.0,0.5,0.0


In [ ]:
km = KMeans(n_clusters= 4, n_init = 2, init = "random", random_state = 42)
km.fit(df2)
y_kmeans = km.predict(df2)
y_kmeans

In [ ]:
df_model = df.copy()

skill_map = {0:"very bad", 1:"bad", 2:"good", 3:"very good"}
df_model['navigating_skills'] = df_model['navigating_skills'].replace(skill_map)

### Step2. Chi-Square of Independence Test

In [ ]:
myCrosstable = pd.crosstab(df_model['cluster'], df_model['navigating_skills'])
myCrosstable

navigating_skills,bad,good,very bad,very good
cluster,,,,
0,21400,152372,3103,128800
1,22820,93572,4177,29517
2,46266,164471,8957,63574
3,11455,97741,1919,59312


In [ ]:
# !pip install scipy

from scipy.stats import chi2_contingency

chiVal, pVal, df, exp = chi2_contingency(myCrosstable)
chiVal, pVal, df, exp

(50596.7559003024,
 0.0,
 9,
 array([[ 34263.13661683, 170795.0525369 ,   6102.36811896,
          94514.4427273 ],
        [ 16823.15244058,  83860.13332806,   2996.25426189,
          46406.45996948],
        [ 31751.53409071, 158275.20386693,   5655.04412308,
          87586.21791928],
        [ 19103.17685188,  95225.61026812,   3402.33349607,
          52695.87938394]]))

In [ ]:
colTotals = myCrosstable.sum()
nCols = len(colTotals)

nCols, colTotals

(4, navigating_skills
 bad          101941
 good         508156
 very bad      18156
 very good    281203
 dtype: int64)

In [ ]:
rowTotals = myCrosstable.sum(axis=1)
nRows = len(rowTotals)

nRows, rowTotals

(4, cluster
 0    305675
 1    150086
 2    283268
 3    170427
 dtype: int64)

In [ ]:
n = sum(rowTotals)
n

909456

## Step3. Posthoc Analyses with Multiple Comparison

In [ ]:
for i in range(nRows):
    for j in range(nCols):
        AdjRes = (myCrosstable.iloc[i,j] - exp[i,j]) / (exp[i,j]*(1-rowTotals[i]/n)*(1-colTotals[j]/n))**0.5
        print(AdjRes)

-90.51071494690036
-82.3628448460867
-47.60039841267087
164.67846476583583
53.696872866023796
55.25166018651436
23.845699989394436
-103.23197409836153
104.1772108908867
28.254356509897498
53.45282165303716
-117.6457996378375
-65.14495322295697
13.61271567756597
-28.49635827282335
38.46792427631717


In [ ]:
phRes = pd.DataFrame(columns=[ 'Group', 'Skills','Adj. Res.'])
for i in range(nRows):
    for j in range(nCols):
        AdjRes = (myCrosstable.iloc[i,j] - exp[i,j]) / (exp[i,j]*(1-rowTotals[i]/n)*(1-colTotals[j]/n))**0.5
        phRes = phRes.append({'Group':myCrosstable.index[i], 'Skills':myCrosstable.columns[j], 'Adj. Res.':AdjRes}, ignore_index=True)

phRes

,Group,Skills,Adj. Res.
0,0,bad,-90.510715
1,0,good,-82.362845
2,0,very bad,-47.600398
3,0,very good,164.678465
4,1,bad,53.696873
5,1,good,55.251660
6,1,very bad,23.845700
7,1,very good,-103.231974
8,2,bad,104.177211
9,2,good,28.254357


In [ ]:
# !pip install scipy
from scipy.stats import norm

In [ ]:
phRes['Sig.'] = 2*(1-norm.cdf(abs(phRes['Adj. Res.'])))
phRes

,Group,Skills,Adj. Res.,Sig.
0,0,bad,-90.510715,0.0
1,0,good,-82.362845,0.0
2,0,very bad,-47.600398,0.0
3,0,very good,164.678465,0.0
4,1,bad,53.696873,0.0
5,1,good,55.251660,0.0
6,1,very bad,23.845700,0.0
7,1,very good,-103.231974,0.0
8,2,bad,104.177211,0.0
9,2,good,28.254357,0.0


In [ ]:
phRes['Adj. Sig.'] = phRes.shape[0]*phRes['Sig.']
phRes

,Group,Skills,Adj. Res.,Sig.,Adj. Sig.
0,0,bad,-90.510715,0.0,0.0
1,0,good,-82.362845,0.0,0.0
2,0,very bad,-47.600398,0.0,0.0
3,0,very good,164.678465,0.0,0.0
4,1,bad,53.696873,0.0,0.0
5,1,good,55.251660,0.0,0.0
6,1,very bad,23.845700,0.0,0.0
7,1,very good,-103.231974,0.0,0.0
8,2,bad,104.177211,0.0,0.0
9,2,good,28.254357,0.0,0.0


In [ ]:
phRes.loc[phRes['Adj. Sig.']> 1, 'Adj. Sig.'] = 1
phRes

,Group,Skills,Adj. Res.,Sig.,Adj. Sig.
0,0,bad,-90.510715,0.0,0.0
1,0,good,-82.362845,0.0,0.0
2,0,very bad,-47.600398,0.0,0.0
3,0,very good,164.678465,0.0,0.0
4,1,bad,53.696873,0.0,0.0
5,1,good,55.251660,0.0,0.0
6,1,very bad,23.845700,0.0,0.0
7,1,very good,-103.231974,0.0,0.0
8,2,bad,104.177211,0.0,0.0
9,2,good,28.254357,0.0,0.0
